In [ ]:
# Install the Anthropic SDK (if not already installed)
!pip install anthropic

# Import libraries
import anthropic
from getpass import getpass
import json

# Set your API key (it'll prompt you securely)
api_key = getpass("Enter your Anthropic API key: ")
client = anthropic.Anthropic(api_key=api_key)
print("API key set!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.8 MB/s eta 0:00:00
Enter your Anthropic API key: ··········
API key set!


In [ ]:
# Define the raw JSON schema for structured output
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "The person's full name"},
        "age": {"type": "integer", "description": "The person's age (must be 0-120)"},
        "hobbies": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of hobbies"
        }
    },
    "required": ["name", "age", "hobbies"],
    "additionalProperties": False
}

# Your prompt (e.g., a bio text)
prompt = "John Doe is a 28-year-old software engineer who loves hiking, coding open-source projects, and playing guitar on weekends."

# Make the beta API call with raw JSON schema
message = client.beta.messages.create(
    model="claude-sonnet-4-5",  # Latest supported model (Sonnet 4.5)
    max_tokens=1024,
    betas=["structured-outputs-2025-11-13"],  # Correct beta header
    messages=[{"role": "user", "content": f"Extract the following info from this bio as structured JSON: {prompt}"}],
    output_format={"type": "json_schema", "schema": json_schema}  # Enforces schema
)

# Parse and print the structured response (pure JSON)
structured_data = json.loads(message.content[0].text)
print("Extracted Structured Data:")
print(json.dumps(structured_data, indent=2))

Extracted Structured Data:
{
  "name": "John Doe",
  "age": 28,
  "hobbies": [
    "hiking",
    "coding open-source projects",
    "playing guitar"
  ]
}


In [ ]:
# Define the JSON schema for order details
json_schema = {
    "type": "object",
    "properties": {
        "order_id": {"type": "string", "description": "Unique order identifier"},
        "customer_name": {"type": "string", "description": "Full name of the customer"},
        "total": {"type": "number", "description": "Total order amount in USD"},
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Item name"},
                    "quantity": {"type": "integer", "description": "Number of units"},
                    "price": {"type": "number", "description": "Price per unit in USD"}
                },
                "required": ["name", "quantity", "price"],
                "additionalProperties": False
            },
            "description": "List of ordered items"
        }
    },
    "required": ["order_id", "customer_name", "total", "items"],
    "additionalProperties": False
}

# Sample email prompt (mimics a real order confirmation)
prompt = """
Subject: Your Order #ORD12345 Confirmation

Dear Jane Smith,

Thank you for your purchase! Your order details are below:

Order ID: ORD12345
Items:
- Wireless Mouse, Qty: 2, Price: $15.99 each
- USB-C Cable, Qty: 1, Price: $9.99
Total: $41.97

Shipping to: 123 Main St, Springfield, IL
Estimated Delivery: Nov 20, 2025

Best,
ShopEasy
"""

# Make the beta API call with raw JSON schema
message = client.beta.messages.create(
    model="claude-sonnet-4-5",  # Latest model (Sonnet 4.5, Nov 2025)
    max_tokens=1024,
    betas=["structured-outputs-2025-11-13"],  # Correct beta header
    messages=[{"role": "user", "content": f"Extract the order details from this email as structured JSON: {prompt}"}],
    output_format={"type": "json_schema", "schema": json_schema}  # Enforces schema
)

# Parse the structured response
structured_data = json.loads(message.content[0].text)
print("Extracted Structured Data:")
print(json.dumps(structured_data, indent=2))

# Save to CSV for production use
import pandas as pd
# Flatten items for CSV (one row per order, items as a list)
order_data = {
    "order_id": structured_data["order_id"],
    "customer_name": structured_data["customer_name"],
    "total": structured_data["total"],
    "items": json.dumps(structured_data["items"])  # Store items as JSON string
}
df = pd.DataFrame([order_data])
df.to_csv("order_output.csv", index=False)
print("\nSaved to 'order_output.csv':")
print(df)

Extracted Structured Data:
{
  "order_id": "ORD12345",
  "customer_name": "Jane Smith",
  "total": 41.97,
  "items": [
    {
      "name": "Wireless Mouse",
      "quantity": 2,
      "price": 15.99
    },
    {
      "name": "USB-C Cable",
      "quantity": 1,
      "price": 9.99
    }
  ]
}

Saved to 'order_output.csv':
   order_id customer_name  total  \
0  ORD12345    Jane Smith  41.97   

                                               items  
0  [{"name": "Wireless Mouse", "quantity": 2, "pr...  


In [ ]:
# Install pandas if not already available (usually pre-installed in Colab)
!pip install pandas

# Import additional libraries
import pandas as pd
from datetime import datetime

# Define the JSON schema for support ticket details
json_schema = {
    "type": "object",
    "properties": {
        "ticket_id": {"type": "string", "description": "Unique ticket identifier (e.g., TICKET-123)"},
        "customer_email": {"type": "string", "description": "Customer's email address"},
        "issue_type": {
            "type": "string",
            "enum": ["Billing", "Technical", "Feature Request", "Other"],
            "description": "Category of the issue"
        },
        "description": {"type": "string", "description": "Brief summary of the issue"},
        "product_id": {
            "type": ["string", "null"],
            "description": "Product ID if mentioned (null if not provided)"
        },
        "priority": {
            "type": "string",
            "enum": ["Low", "Medium", "High"],
            "description": "Priority based on urgency (High for 'urgent', 'crash'; Medium for 'issue'; Low otherwise)"
        }
    },
    "required": ["ticket_id", "customer_email", "issue_type", "description", "priority"],
    "additionalProperties": False
}

# Sample support emails (replace with real emails or load from a file)
support_emails = [
    {
        "ticket_id": "TICKET-001",
        "email": """
From: alice@example.com
Subject: Urgent: App Crashing on Login

Hi Support Team,
The mobile app (Product ID: APP123) keeps crashing when I try to log in. This is urgent as I need access for work. Please help ASAP!
Thanks,
Alice
"""
    },
    {
        "ticket_id": "TICKET-002",
        "email": """
From: bob@example.com
Subject: Question about Billing

Hello,
I was charged twice this month for my subscription. Can you refund one charge? No rush, just want it fixed.
Best,
Bob
"""
    },
    {
        "ticket_id": "TICKET-003",
        "email": """
From: carol@example.com
Subject: Feature Suggestion

Hi,
It’d be great if your app had a dark mode. I don’t have a product ID, just a suggestion for improvement.
Cheers,
Carol
"""
    }
]

# Process emails and collect structured outputs
ticket_data = []
for email_entry in support_emails:
    ticket_id = email_entry["ticket_id"]
    prompt = email_entry["email"]

    # Make the beta API call
    message = client.beta.messages.create(
        model="claude-sonnet-4-5",  # Latest model (Sonnet 4.5, Nov 2025)
        max_tokens=1024,
        betas=["structured-outputs-2025-11-13"],  # Correct beta header
        messages=[{"role": "user", "content": f"Extract support ticket details from this email as structured JSON. Assign a ticket_id of '{ticket_id}' and infer issue_type and priority based on content: {prompt}"}],
        output_format={"type": "json_schema", "schema": json_schema}
    )

    # Parse and store the structured response
    structured_data = json.loads(message.content[0].text)
    ticket_data.append(structured_data)

# Save to CSV with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"tickets_{timestamp}.csv"
df = pd.DataFrame(ticket_data)
df.to_csv(csv_filename, index=False)

# Display results
print("Extracted Structured Tickets:")
print(json.dumps(ticket_data, indent=2))
print(f"\nSaved to '{csv_filename}':")
print(df)

Extracted Structured Tickets:
[
  {
    "ticket_id": "TICKET-001",
    "customer_email": "alice@example.com",
    "issue_type": "Technical",
    "description": "Mobile app keeps crashing when trying to log in",
    "product_id": "APP123",
    "priority": "High"
  },
  {
    "ticket_id": "TICKET-002",
    "customer_email": "bob@example.com",
    "issue_type": "Billing",
    "description": "Customer was charged twice this month for subscription and requests refund for duplicate charge",
    "product_id": null,
    "priority": "Medium"
  },
  {
    "ticket_id": "TICKET-003",
    "customer_email": "carol@example.com",
    "issue_type": "Feature Request",
    "description": "Request for dark mode feature in the app",
    "product_id": null,
    "priority": "Low"
  }
]

Saved to 'tickets_20251115_144855.csv':
    ticket_id     customer_email       issue_type  \
0  TICKET-001  alice@example.com        Technical   
1  TICKET-002    bob@example.com          Billing   
2  TICKET-003  carol@examp